# Strategy 2 membrane segmentation

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import cv2
# model_type='cyto' or 'nuclei' or 'cyto2' OR 'cyto3'
model = models.Cellpose(model_type='cyto')

from tifffile import imwrite
from matplotlib import image

import time
import cv2
from csbdeep.utils import normalize
import glob

In [2]:
path = r'\\store\department\gene\chien_data\Lab\Data_and_Analysis\Tsai-Ying_Chen\TYC070_EC546_TILs_72hr_20250328\72hr_recording_20250328at132435'

WL = True

Blue_channel = False

diameter_1 = 40
diameter_2 = 200

if WL and Blue_channel:
    raise ValueError("Only one of WL or Blue_channel can be True at a time.")

if WL:
    path1 = path + '\\WhiteLight'; 
    path2 = path + f'\\WhiteLight_segm_img_cellpose_d{diameter_1}'
    path3 = path + f'\\WhiteLight_segm_img_cellpose_d{diameter_2}'
    path4 = path + f'\\WhiteLight_segm_img_cellpose_combined_d{diameter_1}_d{diameter_2}'

elif Blue_channel:
    path1 = path + '\\460nm'; 
    path2 = path + f'460nm_segm_img_cellpose_d{diameter_1}'
    path3 = path + f'460nm_segm_img_cellpose_d{diameter_2}'

if not os.path.isdir(path1):
    print("Where is the Whitelight or Blue 460nm channel?")
if not os.path.isdir(path2):
    os.mkdir(path2)
if not os.path.isdir(path3):
    os.mkdir(path3)
if not os.path.isdir(path4):
    os.mkdir(path4)


In [3]:
Timelaps_path_list_WL = glob.glob(path1 + "\\*.tif")

WL_frame_ROI_name =Timelaps_path_list_WL[0].split(path1)
WL_frame_ROI_name = WL_frame_ROI_name[1]
WL_frame_ROI_name = WL_frame_ROI_name.split("\\")
WL_frame_ROI_name = WL_frame_ROI_name[1]

WL_ROI_number = WL_frame_ROI_name
WL_ROI_number = WL_ROI_number.split('.')
WL_ROI_number = WL_ROI_number[0]
WL_ROI_number = WL_ROI_number.split("ROI")
WL_ROI_number = WL_ROI_number[1]

number_digits_ROIs_WL = len(WL_ROI_number)

WL_frame_number = WL_frame_ROI_name.split("frame")
WL_frame_number = WL_frame_number[0] + "frame"
image_prefix_WL = WL_frame_number

print("Image path of segmented channel is " + path1)
print("Image prefix of segmented channel is " + image_prefix_WL)

Image path of segmented channel is \\store\department\gene\chien_data\Lab\Data_and_Analysis\Tsai-Ying_Chen\TYC070_EC546_TILs_72hr_20250328\72hr_recording_20250328at132435\WhiteLight
Image prefix of segmented channel is 72hr_recording_WL_1ms_WhiteLight_frame


In [4]:
def get_object_sizes(mask):
    """Compute sizes of all objects in a given mask."""
    labeled_mask, num_features = scipy.ndimage.label(mask)
    object_sizes = np.array(scipy.ndimage.sum(mask, labeled_mask, range(1, num_features+1)))
    return labeled_mask, object_sizes

def filter_large_masks(masks_large, masks_small, size_threshold=1.2):
    """
    Remove large masks that have a similar size to small masks.
    A large mask is removed if its size is within `size_threshold` times any small mask.
    """
    labeled_large, large_sizes = get_object_sizes(masks_large)
    _, small_sizes = get_object_sizes(masks_small)

    filtered_large = np.zeros_like(masks_large, dtype=np.uint16)  # New mask for filtered large cells

    for i, large_size in enumerate(large_sizes):
        if all(large_size > size_threshold * small_size for small_size in small_sizes):  
            filtered_large[labeled_large == (i+1)] = i+1  # Keep large mask if it's significantly bigger

    return filtered_large

In [ ]:
numFrames = 61

ROIs = range(0, 1, 1)

for frame in range(0, numFrames):
    
    if frame != 60:
        print(f"this is frame {frame}")
        continue

    t = time.time()

    for ROI in ROIs:

        if ROI != 0:
            continue
        
        imgName = (image_prefix_WL + "%05d_ROI%01d.tif" % (frame, ROI))   
        print(imgName)
        img = image.imread(path1+'\\'+imgName, cv2.IMREAD_UNCHANGED);     
        
        # BKG removal
        #filterSize = (100, 100) 
        #kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize) 
        #img = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel) 
        
        print('[Info] Segmenting Cyto ' + imgName + '...');
        img = cv2.GaussianBlur(img, (15, 15), cv2.BORDER_DEFAULT)
        
        img = normalize(img, 1, 75.0, axis=(0, 1))
        
        channels = [0,0]

        #nucleus ~50 pixels big
        t1 = time.time()
        masks_small, flows, styles, diams = model.eval(img, diameter = diameter_1, channels=channels)        
        imwrite(path2 + '\\' + imgName, masks_small)
        elapsed_1 = time.time() - t1
        print('Segmenting %s_d%d took %d seconds.' % (imgName, diameter_1, elapsed_1))

        #cell ~ 100-150 pixels big
        t2 = time.time()
        masks_large, flows, styles, diams = model.eval(img, diameter = diameter_2, channels=channels)        
        imwrite(path3 + '\\' + imgName, masks_large)
        elapsed_2 = time.time() - t2
        print('Segmenting %s_d%d took %d seconds.' % (imgName, diameter_2, elapsed_2))

        # Ensure the large mask takes precedence in combined mask
        combined_mask = np.copy(masks_large)

        # Where there is a small mask but no large mask, add the small mask
        combined_mask[masks_small > 0] = np.where(combined_mask[masks_small > 0] == 0, masks_small[masks_small > 0], combined_mask[masks_small > 0])

        # Save the final combined mask
        imwrite(path4 + '\\' + imgName , combined_mask)

    elapsed = time.time() - t; print('Segmenting frame-%05d took %d seconds.' % (frame, elapsed))

this is frame 0
this is frame 1
this is frame 2
this is frame 3
this is frame 4
this is frame 5
this is frame 6
this is frame 7
this is frame 8
this is frame 9
this is frame 10
this is frame 11
this is frame 12
this is frame 13
this is frame 14
this is frame 15
this is frame 16
this is frame 17
this is frame 18
this is frame 19
this is frame 20
this is frame 21
this is frame 22
this is frame 23
this is frame 24
this is frame 25
this is frame 26
this is frame 27
this is frame 28
this is frame 29
this is frame 30
this is frame 31
this is frame 32
this is frame 33
this is frame 34
this is frame 35
this is frame 36
this is frame 37
this is frame 38
this is frame 39
this is frame 40
this is frame 41
this is frame 42
this is frame 43
this is frame 44
this is frame 45
this is frame 46
this is frame 47
this is frame 48
this is frame 49
this is frame 50
this is frame 51
this is frame 52
this is frame 53
this is frame 54
this is frame 55
this is frame 56
this is frame 57
this is frame 58
this is

In [7]:
img_li = image.imread(r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Tsai-Ying_Chen\TYC070_EC546_TILs_72hr_20250328\72hr_recording_20250328at132435\WhiteLight_ImgGS_merged\72hr_recording_WL_1ms_WhiteLight_frame00060_ROI0.tif", cv2.IMREAD_UNCHANGED);

combined_mask_li = np.copy(img_li)

combined_mask_li[masks_small > 0] = np.where(combined_mask_li[masks_small > 0] == 0, masks_small[masks_small > 0], combined_mask_li[masks_small > 0])

imwrite(r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Tsai-Ying_Chen\TYC070_EC546_TILs_72hr_20250328\72hr_recording_20250328at132435\WhiteLight_segm_img_cellpose_combined_d40_d200\72hr_recording_WL_1ms_WhiteLight_frame00060_ROI0_Li.tif" , combined_mask_li)

## Seperate small vs large cell clusters better 

In [ ]:
path = r'\\store\department\gene\chien_data\Lab\Data_and_Analysis\Tsai-Ying_Chen\TYC066_EC546_TILs_48hr_20250301\48hr_record_20250301at111238'

WL = True
multi_WL = False

Green = True

# If you want to get the coordinates of control cells(normal cells) instead of MN cells
taggingCtrlCells = False

# If you are going to save the results 
save_results = True

def extract_image_prefix(path, channel_name):
    # Get list of all .tif files in the directory
    path_channel = path + f"\\{channel_name}"
    timelaps_path_list = glob.glob(path_channel + "\\*.tif")

    # Extract the frame ROI name from the first image in the list
    frame_roi_name = timelaps_path_list[0].split(path_channel)
    frame_roi_name = frame_roi_name[1]
    frame_roi_name = frame_roi_name.split("\\")
    frame_roi_name = frame_roi_name[1]
    
    # Extract the ROI number from the frame name
    roi_number = frame_roi_name.split('.')
    roi_number = roi_number[0]
    roi_number = roi_number.split("ROI")
    roi_number = roi_number[1]

    # Count the number of digits in the ROI number
    number_digits_rois = len(roi_number)

    # Extract the frame number (before the word 'frame')
    frame_number = frame_roi_name.split("frame")
    frame_number = frame_number[0] + "frame"
    image_prefix = frame_number

    # Print the result
    print(f"Image path of {channel_name} channel is {path_channel}")
    print(f"Image prefix of {channel_name} channel is {path_channel}\\{image_prefix}\n")

    return path_channel, image_prefix, roi_number, number_digits_rois

def segmentation_stardist(path, channel_name, image_prefix, numROIs, number_digits_rois, frame_interval=1, taggingCtrlCells=False):
    path_channel = path + f"\\{channel_name}"
    path_save_img = path + f"\\{channel_name}_segm_img_stardist"
    path_save_data = path + f"\\{channel_name}_segm_img_stardist_csv"

    para = 1;
    if taggingCtrlCells:
        para = 0.1;

    ROIs = range(0, numROIs, 1)

    frame = 0
    print(ROIs)
    print(frame)
    while frame < numFrames: 
    # for frame in frames:
        
        t = time.time()
        
        for ROI in ROIs:        
            
            # Below here is the check to see if the number of ROI's are there that you want to generate
            if number_digits_rois > 1:
                imgName = (image_prefix + "%05d_ROI%03d.tif" % (frame, ROI))
            else:
                imgName = (image_prefix + "%05d_ROI%01d.tif" % (frame, ROI))
            imgName2 = (path_channel+'\\'+imgName)
            
            isImgExist = os.path.exists(imgName2)
            
            while not isImgExist:            
                time.sleep(10)
                print('[Info] ' + imgName +' not generated yet ...')
                isImgExist = os.path.exists(imgName2)
            
            print('[Info] ' + imgName +' is generated!'); time.sleep(1)
            img = image.imread(path_channel+'\\'+imgName)
            
            img = normalize(img, 1, 99.8, axis=(0, 1))

            print('Segmenting ... ' + imgName)
            imgHeight, imgWidth = img.shape; 
            
            #creates the image called labels and polys
            labels, polys = model.predict_instances(img, prob_thresh = 0.15*para)

            centers = polys["points"] #[y, x]
            centers2 = centers[:,0]*imgWidth + centers[:,1]  
            print('   %d cells found.' % centers2.shape[0])
            imwrite(path_save_img + "\\" + imgName, labels.astype(np.uint16))

            #saves the cell center coordinates into a csv
            np.savetxt(path_save_data +"\\" + image_prefix + "%05d_ROI%01d.csv" % (frame, ROI), centers, delimiter=",")
            
        elapsed = time.time() - t; print('Segmenting frame-%05d took %d seconds.' % (frame, elapsed))
            
        frame += frame_interval;



In [ ]:
if WL:
    path_WL, image_prefix_WL, roi_number_WL, number_digits_rois_WL = extract_image_prefix(path, "WhiteLight")
if Green:
    path_532, image_prefix_GR, roi_number_GR, number_digits_rois_GR = extract_image_prefix(path, "532nm")



In [ ]:
# Function to check overlap and size of clusters
def check_overlap_and_filter(mask1, mask2, threshold=0.5):
    """ Check if there is significant overlap between two masks.
        mask1 and mask2 are binary masks (0: background, 1: cell/cluster).
        threshold: Percentage of overlap required to keep the cluster.
    """
    intersection = np.logical_and(mask1, mask2)
    overlap_ratio = np.sum(intersection) / np.sum(mask2)  # Ratio of overlap to cluster size
    return overlap_ratio > threshold

In [ ]:
numFrames = 1

ROIs = range(0, 1, 1)

for frame in range(0, numFrames):

    t = time.time()

    for ROI in ROIs:
        ROI = 33
        #generation WL clusters mask
        imgName_WL = (image_prefix_WL + "%05d_ROI%01d.tif" % (frame, ROI))
        imgName_GR = (image_prefix_GR + "%05d_ROI%01d.tif" % (frame, ROI))

        img_WL = image.imread(path_WL+'\\'+ imgName_WL, cv2.IMREAD_UNCHANGED);     
        
        print('[Info] Segmenting Cyto ' + imgName_WL + '...')
        img_WL = cv2.GaussianBlur(img_WL, (15, 15), cv2.BORDER_DEFAULT)
        
        img_WL = normalize(img_WL, 1, 75.0, axis=(0, 1))
        
        channels = [0,0]

        t1 = time.time()
        diameter_cluster = 250
        masks_large_cluster, flows, styles, diams = model.eval(img_WL, diameter=diameter_cluster, channels=channels)
        path_save_large_cluster = path + f'\\WhiteLight_segm_img_cellpose_d{diameter_cluster}'
        if not os.path.isdir(path_save_large_cluster):
            os.mkdir(path_save_large_cluster)

        imwrite(path_save_large_cluster + '\\' + imgName_WL, masks_large_cluster)
        elapsed_1 = time.time() - t1
        print('Segmenting %s_d%d took %d seconds.' % (imgName_WL, diameter_cluster, elapsed_1))

        #grabbing single cell images
        img_nucleus = image.imread(path + '\\532nm_segm_img_stardist' + '\\' + imgName_GR)
        img_immune = image.imread(path + '\\WhiteLight_segm_img_stardist' + '\\' + imgName_WL)

        # Initialize an empty mask for the final result
        final_mask = np.zeros_like(img_immune)

        # Process the large clusters: only keep clusters with enough overlap with type 2 single cells
        for cluster_label in np.unique(masks_large_cluster):
            if cluster_label == 0:  # Skip background label
                continue
            cluster_mask = (masks_large_cluster == cluster_label)

            # Check if the current cluster has overlap with type 2 single cells
            overlap_with_type2 = check_overlap_and_filter(img_nucleus, cluster_mask, threshold=0.5)
            
            if overlap_with_type2:
                # If the cluster is valid, keep it
                final_mask = np.logical_or(final_mask, cluster_mask)
        
        # Remove type 1 cells that overlap with type 2 (either single cells or clusters)
        overlap_with_type1 = np.logical_and(img_immune, final_mask)
        final_mask = np.logical_or(final_mask, np.logical_not(overlap_with_type1))  # Keep final mask but remove type 1 overlap

        # Optionally: Visualize or save final mask
        final_mask_path = path + "\\Final_mask"
        if not os.path.isdir(final_mask_path):
            os.mkdir(final_mask_path)
        imwrite(final_mask_path + "\\" + imgName_WL, final_mask)



    elapsed = time.time() - t; print('Segmenting frame-%05d took %d seconds.' % (frame, elapsed))

In [ ]:
# Initialize an empty mask for the final result
final_mask = np.zeros_like(img_immune)

# Process the large clusters: only keep clusters with enough overlap with type 2 single cells
for cluster_label in np.unique(masks_large_cluster):
    if cluster_label == 0:  # Skip background label
        continue
    cluster_mask = (masks_large_cluster == cluster_label)

    # Check if the current cluster has overlap with type 2 single cells
    overlap_with_type2 = check_overlap_and_filter(img_nucleus, cluster_mask, threshold=0.5)
    
    if overlap_with_type2:
        # If the cluster is valid, keep it
        final_mask = np.logical_or(final_mask, cluster_mask)

# Remove type 1 cells that overlap with type 2 (either single cells or clusters)
overlap_with_type1 = np.logical_and(img_immune, final_mask)
final_mask = np.logical_or(final_mask, np.logical_not(overlap_with_type1))  # Keep final mask but remove type 1 overlap


# Saving env+log code

In [ ]:
import sys
import subprocess
from IPython import get_ipython

# Define log file
log_file_name = "notebook_log.txt"
log_file_path = path + "\\" + log_file_name

# Capture Python version & environment details
python_version = sys.version
kernel_name = get_ipython().__class__.__name__
conda_env = subprocess.run("conda info --envs", shell=True, capture_output=True, text=True).stdout
installed_packages = subprocess.run("pip freeze", shell=True, capture_output=True, text=True).stdout

# Save everything to log file
with open(log_file_path, "w") as f:
    f.write("\n\n# Environment Details:\n")
    f.write(f"Python Version: {python_version}\n")
    f.write(f"Kernel: {kernel_name}\n")
    f.write(f"Conda Environments:\n{conda_env}\n")
    f.write(f"Installed Packages:\n{installed_packages}\n")
    f.write(f"Used Strategy2_membrane_segmentation.ipynb located in \n\\store\\department\\gene\\chien_data\\Lab\\Data_and_Analysis\\Wilco van Nes\\Code_plugins\\Segmentation ")
print(f"Notebook execution log with environment details saved to {log_file_path}")


## Single file use

In [ ]:
img = image.imread(r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Tsai-Ying_Chen\TYC069_EC546_TILs_48hr_20250312\48hr_record__20250312at111801\460nm\48hr_record__50B_1500ms_frame00000_ROI10.tif", cv2.IMREAD_UNCHANGED);     
        
img = cv2.GaussianBlur(img, (15, 15), cv2.BORDER_DEFAULT)

img = normalize(img, 1, 75.0, axis=(0, 1))

channels = [0,0]

#cell ~ 100-150 pixels big
# masks_large, flows, styles, diams = model.eval(img, diameter = 50, channels=channels)
masks_large, flows, styles, diams = model.eval(img, diameter = 50, channels=channels)                
# Randomize labels for better visualization
unique_labels = np.unique(masks_large)
unique_labels = unique_labels[unique_labels != 0]  # exclude background

random_labels = np.random.permutation(unique_labels) + 1  # make sure labels are non-zero
label_map = {old: new for old, new in zip(unique_labels, random_labels)}

# Apply remapping
masks_random = np.zeros_like(masks_large)
for old_label, new_label in label_map.items():
    masks_random[masks_large == old_label] = new_label

# Save the new labeled image
imwrite(r"\\store\department\gene\chien_data\Lab\Labmembers\Wilco_van_Nes\Thesis\Figures\Experiment 4C\48hr_record__50B_1500ms_frame00000_ROI10_seg_random4.tif", masks_random.astype(np.uint16))
